In [ ]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import datetime, time
import pandas as pd
import openpyxl
from apple_fiscal_year.fiscal_functions import AppleFiscalYear
import mysql.connector as mysql
from sqlalchemy import create_engine
from tqdm import tqdm
from typing import IO, Iterator


In [1]:

total_dic = {}
total_increment = 0


def iter_excel_openpyxl(file: IO[bytes]) -> Iterator[dict[str, object]]:
    workbook = openpyxl.load_workbook(file)

    #print(workbook.sheetnames)

    rows = workbook['Hero Products Data'].rows

    looping_gen = 3
    while(looping_gen):
        next(rows)
        looping_gen = looping_gen - 1

    headers = [str(cell.value) for cell in next(rows)]
    for row in rows:
        yield dict(zip(headers, (cell.value for cell in row)))

loop = 4000

with open('attachments/NSO Data - Rolling Daily TOTAL Sales.xlsx', 'rb') as f:

    rows = iter_excel_openpyxl(f)
    i= 0

    while i < loop:
        try:
            row = next(rows)
            print(row)
            total_dic[total_increment] = row
            total_increment+=1
            #break
            i += 1
        except:
            print("Termination")
            break

#print(total_dic)

NameError: name 'IO' is not defined

In [ ]:
connection = mysql.connect(user='root',
                           password='softzino',
                           host='127.0.0.1',
                           database='softzino')

connection.is_connected()

True

In [2]:
def creating_one_row(obj):

    fiscal_year = AppleFiscalYear()

    my_fiscal_date = obj['Fiscal Day'].date()

    my_fiscal_date = str(my_fiscal_date)

    year = fiscal_year.fiscal_year(my_fiscal_date)

    week = fiscal_year.fiscal_week(my_fiscal_date)

    period = fiscal_year.fiscal_period(my_fiscal_date)

    quarter = fiscal_year.fiscal_quarter(my_fiscal_date)

    day = fiscal_year.fiscal_day(my_fiscal_date)

    now_time = datetime.datetime.now()

    store_num = obj['Org Level 5 (Code)']

    product_type = obj['FPH Level 1 (Name)']

    billing_units = obj['Billing Units']

    billing_amt_usd = obj['Billing Amt USD']

    created_at = datetime.datetime.now()

    created_by = "'CronJob'"

    query = '('
    query += f" '{store_num}', '{my_fiscal_date}', '{product_type}', {billing_units}, {billing_amt_usd}, '{created_at}', {created_by}, {int(day['number'])}, {week['number']}, {period['number']}, {quarter['number']}, {year['number']}"
    query += ')'

    main_query = f"INSERT INTO ma_sap_product_figures (store_num, date, product_type, billing_units, billing_amt_usd, created_at, created_by, fiscal_day, fiscal_week, fiscal_period, fiscal_quarter, fiscal_year) VALUES {query}"

    print(main_query)

    cursor = connection.cursor()
    cursor.execute(main_query)


for i in range(len(total_dic)):
    creating_one_row(total_dic[i])

connection.commit()


NameError: name 'connection' is not defined